# جلسه ۱۵: خلاصه‌سازی، ترجمه و بازنویسی متن Word با هوش مصنوعی


🎯 **هدف:**  
در این جلسه با کمک هوش مصنوعی (مدل‌های آماده از Transformers) متن یک فایل Word را تحلیل کرده، آن را خلاصه، ترجمه یا بازنویسی می‌کنیم.

🛠️ **سرفصل‌ها:**  
1. آپلود فایل Word  
2. استخراج متن  
3. انتخاب نوع پردازش (خلاصه‌سازی، ترجمه، بازنویسی)  
4. اجرای پردازش با هوش مصنوعی


## 1. نصب کتابخانه‌های مورد نیاز

In [ ]:
!pip install python-docx transformers --quiet

## 2. آپلود فایل Word و استخراج متن

In [ ]:
from google.colab import files
from docx import Document

uploaded = files.upload()
text = ""

for fn in uploaded:
    if fn.endswith(".docx"):
        doc = Document(fn)
        text = "\n".join([p.text for p in doc.paragraphs])
        print("✏️ متن استخراج‌شده:")
        print(text[:500])
        break

if not text:
    print("⚠️ فایل Word معتبر یافت نشد.")

## 3. انتخاب عملیات هوشمند

In [ ]:
print("📌 انتخاب نوع پردازش:")
print("1 - خلاصه‌سازی")
print("2 - ترجمه به انگلیسی")
print("3 - بازنویسی رسمی‌تر")

choice = input("شماره عملیات را وارد کنید: ")


## 4. پردازش متن با هوش مصنوعی

In [ ]:
from transformers import pipeline

if choice == "1":
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    result = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    print("\n📄 خلاصه متن:")
    print(result)

elif choice == "2":
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fa-en")
    result = translator(text[:1000])[0]['translation_text']
    print("\n🌍 ترجمه به انگلیسی:")
    print(result)

elif choice == "3":
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fa-en")
    model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fa-en")
    inputs = tokenizer(text[:500], return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=150)
    reformulated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n🪶 بازنویسی رسمی:")
    print(reformulated)

else:
    print("⛔ عملیات نامعتبر.")